# 从零开始实现线性回归

到这里，相信亲爱的读者已经理解了线性回归背后的核心概念，现在我们可以着手通过代码实现线性回归。在这一章节，我们将从零开始实现整个方法， 包括数据传输、模型、损失函数、梯度下降优化器。其实先进的深度学习框架可以基本上自动完成整个工作，但从零开始实现以上这些内容仅仅是为了保证你真的知道你在做什么。并且，当需要自定义模型，定义自己的卷积层或者损失函数等时，如果理解背后的故事那么这些一定是件容易的事。在这一章节，我们只涉及到NDArray和GradientCollector。之后，我们还会介绍一些得益于DJL独到之处的紧凑运行方式。让我们先引入一些必要的包。

In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.7.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26

%maven ai.djl.mxnet:mxnet-engine:0.7.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-b

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-jsplot</artifactId>
    <version>0.38.1</version>
</dependency>

In [ ]:
%load ../utils/plot-utils

In [ ]:
import ai.djl.Device;
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.index.*;
import ai.djl.ndarray.types.DataType;

import tech.tablesaw.api.*;
import tech.tablesaw.plotly.api.*;
import tech.tablesaw.plotly.components.*;

## 生成数据集

为了让接下来的内容更简单，我们要根据一个加入噪声的线性模型创建一个人工的数据集。我们的任务则是用我们数据集中有限的示例来还原模型中的参数。我们将保持低纬度数据这样我们可以轻松地将它们可视化。在接下来的代码片段中，我们将生成一个包含１０００个示例的数据集，每个示例均包含2个从标准正态分布中采样的特征。因此，我们的数据集可以被写成$\mathbf{X} \in \mathbb{R}^{1000 \times 2}$。
生成我们数据的真实参数将是$\mathbf{w}=[2,-3.4]^{\top}$和$\mathbf{ｂ}＝{4.2}$，我们的复合标签将根据下面带有噪音级数$\epsilon$的线性模型被赋值：

$\mathbf{y}=\mathbf{X} \mathbf{w}+b+\epsilon$

你可以将$\epsilon$视为获取特征和标签时的潜在测量错误。假设，此时标准假设成立，那么$\epsilon$服从正态分布且均值为0。为了是我们的问题简化，我们设它的标准偏差为$\mathbf{0.01}$.在接下来的代码中，我们生成了数据集：

class DataPoints {
    private NDArray X, y;
    public DataPoints(NDArray X, NDArray y) {
        this.X = X;
        this.y = y;
    }

    public NDArray getX() {
        return X;
    }

    public NDArray getY() {
        return y;
    }
}

// Generate y = X w + b + noise
public DataPoints syntheticData(NDManager manager, NDArray w, float b, int numExamples) {
    NDArray X = manager.randomNormal(new Shape(numExamples, w.size()));
    NDArray y = X.dot(w).add(b);
    // Add noise
    y = y.add(manager.randomNormal(0, 0.01f, y.getShape(), DataType.FLOAT32, Device.defaultDevice()));
    return new DataPoints(X, y);
}

NDManager manager = NDManager.newBaseManager();

NDArray trueW = manager.create(new float[]{2, -3.4f});
float trueB = 4.2f;

DataPoints dp = syntheticData(manager, trueW, trueB, 1000);
NDArray features = dp.getX();
NDArray labels = dp.getY();

注意，特征中的每一行都包含一个二维的数据点而且标签中每一行也包含一个以为的目标值（一个标量）。

In [ ]:
System.out.printf("features: [%f, %f]\n", features.get(0).getFloat(0), features.get(0).getFloat(1));
System.out.println("label: " + labels.getFloat(0));

通过使用第二个特征［：，１］和标签，生成散点图，我们可以清楚的观察到两者之间的线性关系。

In [ ]:
float[] X = features.get(new NDIndex(":, 1")).toFloatArray();
float[] y = labels.toFloatArray();

Table data = Table.create("Data")
    .addColumns(
        FloatColumn.create("X", X),
        FloatColumn.create("y", y)
    );

ScatterPlot.create("Synthetic Data", data, "X", "y");

## 读取数据集

回想一下，训练模型，包括对数据集进行多次遍历，一次获取一个小批量示例，并用它们更新模型。这一次，我们可以使用ArrayDataset来随机采样数据，并小批量地访问数据。
在接下来的代码中，我们实例化一个ArrayDataset。接着给features, labels, batchSize, and sampling设置一些参数。
而使用dataset.getData，我们可以得到batchSize小批量的大小，每一个组成它的特征和标签。

In [ ]:
import ai.djl.training.dataset.ArrayDataset;
import ai.djl.training.dataset.Batch;

int batchSize = 10;

ArrayDataset dataset = new ArrayDataset.Builder()
                          .setData(features) // Set the Features
                          .optLabels(labels) // Set the Labels
                          .setSampling(batchSize, false) // set the batch size and random sampling to false
                          .build();

一般而言，请注意，我们要使用合适大小的batchSize，这样才能更好地利用GPU的并行化操作优势。由于每个示例都可以被并行地喂进模型，并且每个示例中，损失函数的梯度也可以并行获取，因此GPU使我们在处理数百个示例时所花的时间几乎只比处理单个示例所花费的时间多一点。
为了更加直观，我们来读取并打印第一小批数据中的示例。每个小批量中特征的形状告诉我们小批量的大小和输入特征的数量。同样，通过设定batchSize的大小可以确定小批量中标签的形状。

In [ ]:
for (Batch batch : dataset.getData(manager)) {
    // Call head() to get the first NDArray
    NDArray X = batch.getData().head();
    NDArray y = batch.getLabels().head();
    System.out.println(X);
    System.out.println(y);
    // Don't forget to close the batch!
    batch.close();
    break;
}

当我们运行这个迭代时，将会得到离散的小批量直到所有数据被取尽。虽然运行上述的迭代对于实现目前教学目的是有好处的，但它的效率很低，可能会在实际问题上带来麻烦。比如，现实要求我们将所有数据加载到内存中，并且能够执行大量随机访问。这时，就需要DJL中内置的迭代器，因为它不仅效率更高，而且它可以同时处理存储在文件中的数据并通过数据流的方式喂进模型。

## 初始化模型参数

开始通过梯度下降算法优化模型之前，我们首先需要一些初始化参数。在下面的代码中，通过从均值为0和标准偏差为0.01的正态分布中采样随机数来初始化权重，也就是我们一开始需要的初始化参数，并将偏差b设置为0。

In [ ]:
NDArray w = manager.randomNormal(0, 0.01f, new Shape(2, 1), DataType.FLOAT32, Device.defaultDevice());
NDArray b = manager.zeros(new Shape(1));
NDList params = new NDList(w, b);

现在我们已将参数初始化，下一步则是更新它们使它们足够适应我们的数据。每一次更新都需要根据现在的参数得到损失函数的梯度（一个多维度的导数）。有了个这个梯度，我们才可以在损失减小的方向更新每一个参数。

由于没有人愿意显式计算梯度（繁琐且容易出错），因而使用自动微分来计算梯度。请通过sec_gradcollector查看相关细节。回顾涉及autograd的章节，为了使GradientCollector知道它应该为了下一步更新参数而储存梯度，需要调用attachGradient（）函数，来分配内存以存储我们计划将要获得的梯度。

## 定义模型

下一步，我们必须定义模型，包括它的输入和与它相关的输出参数。回顾一下计算线性模型输出的章节，我们只需要示例$\mathbf{X}$的矩阵向量点积和模型权重$\mathbf{w}$，然后将偏置$\mathbf{ｂ}$添加到每个实例中。请注意，下面代码块中的X.dot(w)是一个向量，ｂ是个标量。回忆一下，当我们将一个向量和标量相加时，向量中的每一个成员都将与此标量相加。

In [ ]:
// Saved in Training.java for later use
public NDArray linreg(NDArray X, NDArray w, NDArray b) {
    return X.dot(w).add(b);
}

## 定义损失函数

因为更新我们的模型需要损失函数的梯度，我们应该首先定义损失函数。这里我们将用到之前章节中提到过的平方损失函数。在这次运行中，我们需要转换正值ｙ到预测值的形状yHat。以下函数返回的结果也将与yHat形状相同。

In [ ]:
// Saved in Training.java for later use
public NDArray squaredLoss(NDArray yHat, NDArray y) {
    return (yHat.sub(y.reshape(yHat.getShape()))).mul
        ((yHat.sub(y.reshape(yHat.getShape())))).div(2);
}

## 定义优化算法

正如我们之前章节中探讨的一样，线性回归有一个封闭形式的解决方案。然而，本书不是一本关于线性回归的书而是一本关于深度学习的书。由于本书介绍的其他模型都不能被解析，借此机会，我们将想你介绍你的第一个工作范例，随机梯度下降stochastic gradient descent (SGD)。

在每一步中，使用从数据集中随机抽取的一批，我们将估算相对于参数的损失梯度。接着，我们会在损失减少的方向更新参数。回忆sec_gradcollector，在调用 backward()之后，每一个参数，它的梯度将被储存在 param.getGradient()中。接下来的代码，使用了SGD更新，通过给定一系列参数，一个学习率，一个batch的大小。其中学习率决定更新步长。因为我们的损失是经过一整个batch示例损失的总和，所以需要通过batchSize归一化步长，因此，通常步长的大小在很大程度上不取决于对batchSize的选择。

In [ ]:
// Saved in Training.java for later use
public static void sgd(NDList params, float lr, int batchSize) {
    for (int i = 0; i < params.size(); i++) {
        NDArray param = params.get(i);
        // Update param
        // param = param - param.gradient * lr / batchSize
        param.subi(param.getGradient().mul(lr).div(batchSize));
    }
}

## 训练